In [ ]:
"""
Created on Wed Jun 09 14:36 2021

Prepare bedrock slope for use in the neural network

Author: @claraburgard

"""

In [ ]:
import numpy as np
import xarray as xr
from tqdm.notebook import trange, tqdm
import basal_melt_neural_networks.data_formatting as dfmt


READ IN DATA

In [ ]:
nemo_run = 'OPM006'
inputpath_data='/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/NEMO_eORCA025.L121_'+nemo_run+'_ANT_STEREO/'
inputpath_mask = '/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/ANTARCTICA_IS_MASKS/nemo_5km_'+nemo_run+'/'

In [ ]:
file_isf_orig = xr.open_dataset(inputpath_mask+'nemo_5km_isf_masks_and_info_and_distance_new.nc')
nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
large_isf = file_isf_nonnan['Nisf'].where(file_isf_nonnan['isf_area_here'] >= 2500, drop=True)
file_isf = file_isf_nonnan.sel(Nisf=large_isf)

In [ ]:
map_lim = [-3000000,3000000]
file_mask_orig = xr.open_dataset(inputpath_data+'other_mask_vars_Ant_stereo.nc')
file_mask_orig_cut = dfmt.cut_domain_stereo(file_mask_orig, map_lim, map_lim)
file_other = xr.open_dataset(inputpath_data+'corrected_draft_bathy_isf.nc')#, chunks={'x': chunk_size, 'y': chunk_size})
file_other_cut = dfmt.cut_domain_stereo(file_other, map_lim, map_lim)
file_conc = xr.open_dataset(inputpath_data+'isfdraft_conc_Ant_stereo.nc')
file_conc_cut = dfmt.cut_domain_stereo(file_conc, map_lim, map_lim)

In [ ]:
file_bed_corr = file_other_cut['corrected_isf_bathy']
file_bed_orig = file_mask_orig_cut['bathy_metry']
file_draft = file_other_cut['corrected_isfdraft'] 
file_isf_conc = file_conc_cut['isfdraft_conc']
file_bed_goodGL = file_bed_orig.where(file_draft < file_bed_orig,file_bed_corr)


In [ ]:
kisf=66
dx = file_isf.x[2] - file_isf.x[1]
dy = file_isf.y[2] - file_isf.y[1]
lonslope, latslope = slope_zonal_merid(kisf, file_isf, ice_draft_neg, dx, dy)

In [ ]:
bb_merid_slope = None
bb_zonal_slope = None

ice_merid_slope = None
ice_zonal_slope = None

dx = file_isf.x[2] - file_isf.x[1]
dy = file_isf.y[2] - file_isf.y[1]

isf_area_map = file_isf['ISF_mask'].where(file_isf['ISF_mask']>1).copy()
entry_depth_map = file_isf['ISF_mask'].where(file_isf['ISF_mask']>1).copy()

for kisf in tqdm(file_isf.Nisf):
    #print(kisf.values)
    bb_lonslope, bb_latslope = dfmt.slope_zonal_merid(kisf, file_isf, -1*file_bed_goodGL, dx, dy)
    #print('here1')
    ice_lonslope, ice_latslope = dfmt.slope_zonal_merid(kisf, file_isf, -1*file_draft, dx, dy)
    #print('here2')
    if bb_merid_slope is None:
        bb_merid_slope = bb_lonslope
        bb_zonal_slope = bb_latslope
        ice_merid_slope = ice_lonslope
        ice_zonal_slope = ice_latslope
    else:
        bb_merid_slope = bb_merid_slope.combine_first(bb_lonslope)
        bb_zonal_slope = bb_zonal_slope.combine_first(bb_latslope)
        ice_merid_slope = ice_merid_slope.combine_first(ice_lonslope)
        ice_zonal_slope = ice_zonal_slope.combine_first(ice_latslope)
     
    #print('here3')
    isf_area_map = isf_area_map.where(file_isf['ISF_mask']!=kisf, file_isf['isf_area_here'].sel(Nisf=kisf).drop('Nisf'))
    #print('here4')
    entry_depth_map = entry_depth_map.where(file_isf['ISF_mask']!=kisf, file_isf['front_bot_depth_max'].sel(Nisf=kisf).drop('Nisf'))
    #print('here5')
        
bb_merid_slope_smooth = bb_merid_slope.reindex_like(file_isf)
bb_zonal_slope_smooth = bb_zonal_slope.reindex_like(file_isf)
ice_merid_slope_smooth = ice_merid_slope.reindex_like(file_isf)
ice_zonal_slope_smooth = ice_zonal_slope.reindex_like(file_isf)


dataset_res_merid_bed = bb_merid_slope_smooth.to_dataset(name='slope_bed_lon')
dataset_res_zonal_bed = bb_zonal_slope_smooth.to_dataset(name='slope_bed_lat')
dataset_res_merid_ice = ice_merid_slope_smooth.to_dataset(name='slope_ice_lon')
dataset_res_zonal_ice = ice_zonal_slope_smooth.to_dataset(name='slope_ice_lat')


dataset_res = xr.merge([dataset_res_merid_bed,dataset_res_zonal_bed,dataset_res_merid_ice,dataset_res_zonal_ice,
                       isf_area_map.to_dataset(name='isf_area'),
                       entry_depth_map.to_dataset(name='entry_depth_max')])
#dataset_res.to_netcdf(inputpath_mask+'nemo_5km_slope_info_bedrock_draft_latlon.nc','w')

In [ ]:
dataset_res.to_netcdf(inputpath_mask+'nemo_5km_slope_info_bedrock_draft_latlon.nc','w')

######## TRYING OUT THE CONVERSION FORMULA

In [ ]:
dataset_res_2 = xr.open_dataset(inputpath_mask+'nemo_5km_slope_info_bedrock_draft.nc')

ice_draft_neg = -1*file_bed_goodGL
kisf = 31

draft_isf = ice_draft_neg.where(file_isf['ISF_mask'] == kisf, drop=True)

dr = np.pi/180.
#deltaphi = 90. - GEddxx !Geddxx= 90° in my config
deltaphi = 0

phi = (-1)*(file_isf.longitude+deltaphi)*dr #to turn positively
cphi = np.cos( phi )
sphi = np.sin( phi )
#
ux = cphi*dataset_res['slope_bed_x'].values + sphi*(-1)*dataset_res['slope_bed_y'].values #because the y-axis is in the other direction
vy = - sphi*dataset_res['slope_bed_x'].values + cphi*(-1)*dataset_res['slope_bed_y'].values #because the y-axis is in the other direction
u_lon = xr.DataArray(ux, coords=dataset_res.coords).transpose('y','x').rename('slope_lon')
v_lat = xr.DataArray(vy, coords=dataset_res.coords).transpose('y','x').rename('slope_lat')

# also, to double-check : https://www.whoi.edu/marine/ndsf/utility/NDSFutility.js

In [ ]:
dataset_res2 = xr.open_dataset(inputpath_mask+'nemo_5km_slope_info_bedrock_draft.nc')


In [ ]:
kisf=10
keyword='ice'

In [ ]:
dataset_res['slope_'+keyword+'_lon'].where(file_isf['ISF_mask'] == kisf, drop=True).plot()

In [ ]:
(-1*dataset_res2['slope_'+keyword+'_y'].where(file_isf['ISF_mask'] == kisf, drop=True)).plot()

In [ ]:
dataset_res['slope_'+keyword+'_lat'].where(file_isf['ISF_mask'] == kisf, drop=True).plot()

In [ ]:
(dataset_res2['slope_'+keyword+'_x']).where(file_isf['ISF_mask'] == kisf, drop=True).plot()

In [ ]:
dataset_res['slope_bed_y'].where(file_isf['ISF_mask'] == 66, drop=True).isel(x=range(12,15),y=range(12,15))

In [ ]:
kisf = 39 # Abbot

In [ ]:
u_lon.where(file_isf['ISF_mask'] == kisf, drop=True).plot()

In [ ]:
(-1*dataset_res['slope_bed_y'].where(file_isf['ISF_mask'] == kisf, drop=True)).plot()

In [ ]:
v_lat.where(file_isf['ISF_mask'] == kisf, drop=True).plot()

In [ ]:
(dataset_res['slope_bed_x']).where(file_isf['ISF_mask'] == kisf, drop=True).plot()

In [ ]:
kisf = 44 #Fimbul

In [ ]:
u_lon.where(file_isf['ISF_mask'] == kisf, drop=True).plot()

In [ ]:
(dataset_res['slope_bed_x']).where(file_isf['ISF_mask'] == kisf, drop=True).plot()

In [ ]:
v_lat.where(file_isf['ISF_mask'] == kisf, drop=True).plot()

In [ ]:
(-1*dataset_res['slope_bed_y'].where(file_isf['ISF_mask'] == kisf, drop=True)).plot()

In [ ]:
kisf=10

In [ ]:
u_lon.where(file_isf['ISF_mask'] == kisf, drop=True).plot()

In [ ]:
(dataset_res['slope_bed_x']).where(file_isf['ISF_mask'] == kisf, drop=True).plot()

In [ ]:
v_lat.where(file_isf['ISF_mask'] == kisf, drop=True).plot()

In [ ]:
(-1*dataset_res['slope_bed_y'].where(file_isf['ISF_mask'] == kisf, drop=True)).plot()

In [ ]:
# this is the lon, lat stuff in the x and y directions

x_slope_bed_lat = None
x_slope_bed_lon = None
y_slope_bed_lat = None
y_slope_bed_lon = None
x_slope_ice_lat = None
x_slope_ice_lon = None
y_slope_ice_lat = None
y_slope_ice_lon = None

isf_area_map = file_isf['ISF_mask'].where(file_isf['ISF_mask']>1).copy()
entry_depth_map = file_isf['ISF_mask'].where(file_isf['ISF_mask']>1).copy()

for kisf in tqdm(file_isf.Nisf):
    print(kisf.values)
    bb_sl_x_lat, bb_sl_x_lon, bb_sl_y_lat, bb_sl_y_lon = dfmt.slope_in_lon_and_lat_dir(kisf, file_isf, -1*file_bed_goodGL, 'bathy_metry')
    #print('here1')
    ice_sl_x_lat, ice_sl_x_lon, ice_sl_y_lat, ice_sl_y_lon = dfmt.slope_in_lon_and_lat_dir(kisf, file_isf, -1*file_draft, 'corrected_isfdraft')
    #print('here2')
    if x_slope_bed_lat is None:
        x_slope_bed_lat = bb_sl_x_lat
        x_slope_bed_lon = bb_sl_x_lon
        y_slope_bed_lat = bb_sl_y_lat
        y_slope_bed_lon = bb_sl_y_lon
        x_slope_ice_lat = ice_sl_x_lat
        x_slope_ice_lon = ice_sl_x_lon
        y_slope_ice_lat = ice_sl_y_lat
        y_slope_ice_lon = ice_sl_y_lon
    else:
        x_slope_bed_lat = x_slope_bed_lat.combine_first(bb_sl_x_lat)
        x_slope_bed_lon = x_slope_bed_lon.combine_first(bb_sl_x_lon)
        y_slope_bed_lat = y_slope_bed_lat.combine_first(bb_sl_y_lat)
        y_slope_bed_lon = y_slope_bed_lon.combine_first(bb_sl_y_lon)
        
        x_slope_ice_lat = x_slope_ice_lat.combine_first(ice_sl_x_lat)
        x_slope_ice_lon = x_slope_ice_lon.combine_first(ice_sl_x_lon)
        y_slope_ice_lat = y_slope_ice_lat.combine_first(ice_sl_y_lat)
        y_slope_ice_lon = y_slope_ice_lon.combine_first(ice_sl_y_lon)        
    #print('here3')
    isf_area_map = isf_area_map.where(file_isf['ISF_mask']!=kisf, file_isf['isf_area_here'].sel(Nisf=kisf).drop('Nisf'))
    #print('here4')
    entry_depth_map = entry_depth_map.where(file_isf['ISF_mask']!=kisf, file_isf['front_bot_depth_max'].sel(Nisf=kisf).drop('Nisf'))
    #print('here5')
        
x_slope_bed_lat_smooth = x_slope_bed_lat.reindex_like(file_isf)
x_slope_bed_lon_smooth = x_slope_bed_lon.reindex_like(file_isf)
y_slope_bed_lat_smooth = y_slope_bed_lat.reindex_like(file_isf)
y_slope_bed_lon_smooth = y_slope_bed_lon.reindex_like(file_isf)

x_slope_ice_lat_smooth = x_slope_ice_lat.reindex_like(file_isf)
x_slope_ice_lon_smooth = x_slope_ice_lon.reindex_like(file_isf)
y_slope_ice_lat_smooth = y_slope_ice_lat.reindex_like(file_isf)
y_slope_ice_lon_smooth = y_slope_ice_lon.reindex_like(file_isf)


dataset_res_x_lat_bed = x_slope_bed_lat_smooth.to_dataset(name='slope_bed_x_lat')
dataset_res_x_lon_bed = x_slope_bed_lon_smooth.to_dataset(name='slope_bed_x_lon')
dataset_res_y_lat_bed = y_slope_bed_lat_smooth.to_dataset(name='slope_bed_y_lat')
dataset_res_y_lon_bed = y_slope_bed_lon_smooth.to_dataset(name='slope_bed_y_lon')

dataset_res_x_lat_ice = x_slope_ice_lat_smooth.to_dataset(name='slope_ice_x_lat')
dataset_res_x_lon_ice = x_slope_ice_lon_smooth.to_dataset(name='slope_ice_x_lon')
dataset_res_y_lat_ice = y_slope_ice_lat_smooth.to_dataset(name='slope_ice_y_lat')
dataset_res_y_lon_ice = y_slope_ice_lon_smooth.to_dataset(name='slope_ice_y_lon')

dataset_res = xr.merge([dataset_res_x_lat_bed,dataset_res_x_lon_bed,dataset_res_y_lat_bed,dataset_res_y_lon_bed,
                       dataset_res_x_lat_ice,dataset_res_x_lon_ice,dataset_res_y_lat_ice,dataset_res_y_lon_ice,
                       isf_area_map.to_dataset(name='isf_area'),
                       entry_depth_map.to_dataset(name='entry_depth_max')])
dataset_res.to_netcdf(inputpath_mask+'nemo_5km_slope_info_bedrock_draft_latlon.nc','w')
#bedrock_slope_smooth.to_dataset(name='bedrock_slope').to_netcdf(inputpath_mask+'nemo_5km_bedrock_slope.nc','w')

################# OLD SCRIPTS

In [ ]:
# aboslute slope in both directions (like for plume, local slope option)
dx = file_isf.x[2] - file_isf.x[1]
dy = file_isf.y[2] - file_isf.y[1]

bedrock_slope = None
for kisf in tqdm(file_isf.Nisf):
    #print(kisf.values)
    bb_sl = compute_alpha_appenB(kisf, file_isf, -1*file_bed_goodGL, abs(dx), abs(dy))
    if bedrock_slope is None:
        bedrock_slope = bb_sl
    else:
        bedrock_slope = bedrock_slope.combine_first(bb_sl)
bedrock_slope_smooth = bedrock_slope.reindex_like(file_isf)
bedrock_slope_smooth.to_dataset(name='bedrock_slope').to_netcdf(inputpath_mask+'nemo_5km_bedrock_slope.nc','w')

In [ ]:
ice_draft_neg = -1*file_bed_goodGL
plume_var_of_int = file_isf
kisf=66

In [ ]:
shift_vars = xr.merge([draft_isf.drop('latitude').drop('longitude'),lat,lon])

In [ ]:
draft_isf = ice_draft_neg.where(plume_var_of_int['ISF_mask'] == kisf, drop=True)
lat = plume_var_of_int['latitude'].where(plume_var_of_int['ISF_mask'] == kisf, drop=True).drop('latitude').drop('longitude')
lon = plume_var_of_int['longitude'].where(plume_var_of_int['ISF_mask'] == kisf, drop=True).drop('latitude').drop('longitude')
shift_vars = xr.merge([draft_isf.drop('latitude').drop('longitude'),lat,lon])

In [ ]:
shift_vars_x_minus = shift_vars.shift(x=-1)
shift_vars_x_plus = shift_vars.shift(x=1)
shift_vars_y_minus = shift_vars.shift(y=-1)
shift_vars_y_plus = shift_vars.shift(y=1)

In [ ]:
shift_vars_test = shift_vars_x_minus - shift_vars
shift_vars_test['longitude']

In [ ]:
# this is the x and y slope only
dx = file_isf.x[2] - file_isf.x[1]
dy = file_isf.y[2] - file_isf.y[1]

x_slope_ice = None
y_slope_ice = None
x_slope_bed = None
y_slope_bed = None

for kisf in tqdm(file_isf.Nisf):
    #print(kisf.values)
    bb_sl_x, bb_sl_y = slope_in_x_and_y_dir(kisf, file_isf, -1*file_bed_goodGL, abs(dx), abs(dy))
    ice_sl_x, ice_sl_y = slope_in_x_and_y_dir(kisf, file_isf, -1*file_draft, abs(dx), abs(dy))
    if x_slope_bed is None:
        x_slope_bed = bb_sl_x
        y_slope_bed = bb_sl_y
        x_slope_ice = ice_sl_x
        y_slope_ice = ice_sl_y
    else:
        x_slope_bed = x_slope_bed.combine_first(bb_sl_x)
        y_slope_bed = y_slope_bed.combine_first(bb_sl_y)
        x_slope_ice = x_slope_ice.combine_first(ice_sl_x)
        y_slope_ice = y_slope_ice.combine_first(ice_sl_y)
        
x_slope_bed_smooth = x_slope_bed.reindex_like(file_isf)
y_slope_bed_smooth = y_slope_bed.reindex_like(file_isf)
x_slope_ice_smooth = x_slope_ice.reindex_like(file_isf)
y_slope_ice_smooth = y_slope_ice.reindex_like(file_isf)

dataset_res_x_bed = x_slope_bed_smooth.to_dataset(name='slope_bed_x')
dataset_res_y_bed = y_slope_bed_smooth.to_dataset(name='slope_bed_y')
dataset_res_x_ice = x_slope_ice_smooth.to_dataset(name='slope_ice_x')
dataset_res_y_ice = y_slope_ice_smooth.to_dataset(name='slope_ice_y')
dataset_res = xr.merge([dataset_res_x_bed,dataset_res_y_bed,dataset_res_x_ice,dataset_res_y_ice])
#dataset_res.to_netcdf(inputpath_mask+'nemo_5km_slope_info_bedrock_draft.nc','w')
#bedrock_slope_smooth.to_dataset(name='bedrock_slope').to_netcdf(inputpath_mask+'nemo_5km_bedrock_slope.nc','w')

######## CHRISTOPH FORMULA - NOT REALLY SURE IF THIS IS RIGHT BUT I COULD TRUST HIM

In [ ]:
dataset_res = xr.open_dataset(inputpath_mask+'nemo_5km_slope_info_bedrock_draft.nc')

dr = np.pi/180.
#deltaphi = 90. - GEddxx !Geddxx= 90° in my config
deltaphi = 0

phi = (-1.) * (file_isf.longitude+deltaphi)*dr
cphi = np.cos( -phi )
sphi = np.sin( -phi )
ux = cphi*dataset_res['slope_bed_x'].values - sphi*dataset_res['slope_bed_y'].values
vy = sphi*dataset_res['slope_bed_x'].values + cphi*dataset_res['slope_bed_y'].values
u_lonlat = xr.DataArray(ux, coords=dataset_res.coords).transpose('y','x')
#u_lonlat = u_lonlat.set_coords(dataset_res.coords)
v_lonlat = xr.DataArray(vy, coords=dataset_res.coords).transpose('y','x')

In [ ]:
u_lonlat.where(file_isf['ISF_mask'] == 66, drop=True).isel(x=range(12,15),y=range(12,15))#.plot()

In [ ]:
dataset_res

In [ ]:
dataset_res['slope_bed_x'].where(file_isf['ISF_mask'] == 66, drop=True).isel(x=range(12,15),y=range(12,15))#.plot()

In [ ]:
file_bed_goodGL.where(file_isf['ISF_mask'] == 66, drop=True).isel(x=range(12,15),y=range(12,15))#.plot()

In [ ]:
file_bed_goodGL.where(file_isf['ISF_mask'] == 66, drop=True).isel(x=range(12,15),y=range(12,15))#.plot()

In [ ]:
dataset_res['slope_bed_y_lon'].where(file_isf['ISF_mask'] == 66, drop=True).isel(x=range(12,15),y=range(12,15))#.plot()

In [ ]:
-5/-0.34

########### PREPARE WEIRD INPUT TO TELL THE ORIENTATION OF THE ICE SHELF

In [ ]:
def diff_GL_IF(kisf, mask_file, dimension, direction):
    GL_mask = mask_file['GL_mask'].where(mask_file['GL_mask'] == kisf, drop=True)
    IF_mask = mask_file['IF_mask'].where(mask_file['IF_mask'] == kisf, drop=True)
    if dimension == 'x':
        if direction == 'minmax':
            diff_GL_IF = GL_mask.x.min() - IF_mask.x.max()
        elif direction == 'maxmin':
            diff_GL_IF = GL_mask.x.max() - IF_mask.x.min()
    elif dimension == 'y':
        if direction == 'minmax':
            diff_GL_IF = GL_mask.y.min() - IF_mask.y.max()
        elif direction == 'maxmin':
            diff_GL_IF = GL_mask.y.max() - IF_mask.y.min()
    return diff_GL_IF

In [ ]:
dx_GL_IF_minmax_map = file_isf['ISF_mask'].where(file_isf['ISF_mask']>1).copy()
dx_GL_IF_maxmin_map = file_isf['ISF_mask'].where(file_isf['ISF_mask']>1).copy()
dy_GL_IF_minmax_map = file_isf['ISF_mask'].where(file_isf['ISF_mask']>1).copy()
dy_GL_IF_maxmin_map = file_isf['ISF_mask'].where(file_isf['ISF_mask']>1).copy()
isf_area_map = file_isf['ISF_mask'].where(file_isf['ISF_mask']>1).copy()

for kisf in tqdm(file_isf.Nisf):
    dx_GL_IF_minmax = diff_GL_IF(kisf, file_isf, 'x', 'minmax')
    dx_GL_IF_minmax_map = dx_GL_IF_minmax_map.where(file_isf['ISF_mask']!=kisf, dx_GL_IF_minmax)
    
    dx_GL_IF_maxmin = diff_GL_IF(kisf, file_isf, 'x', 'maxmin')
    dx_GL_IF_maxmin_map = dx_GL_IF_maxmin_map.where(file_isf['ISF_mask']!=kisf, dx_GL_IF_maxmin)
                                                    
    dy_GL_IF_minmax = diff_GL_IF(kisf, file_isf, 'y', 'minmax')
    dy_GL_IF_minmax_map = dy_GL_IF_minmax_map.where(file_isf['ISF_mask']!=kisf, dy_GL_IF_minmax)

    dy_GL_IF_maxmin = diff_GL_IF(kisf, file_isf, 'y', 'maxmin')
    dy_GL_IF_maxmin_map = dy_GL_IF_maxmin_map.where(file_isf['ISF_mask']!=kisf, dy_GL_IF_maxmin)

    isf_area_map = isf_area_map.where(file_isf['ISF_mask']!=kisf, file_isf['isf_area_here'].sel(Nisf=kisf).drop('Nisf'))
    
dataset_res_dGL_IF = xr.merge([dx_GL_IF_minmax_map.to_dataset(name='dx_GL_IF_minmax'),
                        dx_GL_IF_maxmin_map.to_dataset(name='dx_GL_IF_maxmin'),
                        dy_GL_IF_minmax_map.to_dataset(name='dy_GL_IF_minmax'),
                        dy_GL_IF_maxmin_map.to_dataset(name='dy_GL_IF_maxmin'),
                        isf_area_map.to_dataset(name='isf_area')])
dataset_res_dGL_IF.to_netcdf(inputpath_mask+'nemo_5km_orientation_info.nc','w')

In [ ]:
test_file_bed = file_bed_orig.where(file_draft < file_bed_orig,file_bed_corr)
test_file_bed.where(file_isf['ISF_mask']==66, drop=True).plot()

In [ ]:
test_file_bed = file_bed_orig.where(file_isf['dGL'] >= 10000.,file_bed_corr)
test_file_bed.where(file_isf['ISF_mask']==66, drop=True).plot()

In [ ]:
(test_file_bed - file_bed_orig).where(file_isf['ISF_mask']==10, drop=True).plot()